In [1]:
import numpy as np
import sys
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import interferometer as itf
sys.path.append('../') # Add parent directory to the system path
from boson_sampling_probabilities import output_probability
from direct_decomposition import direct_decomposition, random_unitary
from simulation import run_interferom_simulation

from numeric_truncated_unitaries import numeric_truncated_unitary
from direct_decomposition import round_complex_matrix, compile_unitary, knit_qiskit_circuits

In [2]:
I = itf.Interferometer()
BS1 = itf.Beamsplitter(1, 2, theta=np.pi/4, phi=0)
BS2 = itf.Beamsplitter(2, 3, theta=np.pi/3, phi=0)
BS3 = itf.Beamsplitter(1, 2, theta=np.pi/3, phi=0)
I.add_BS(BS1)
I.add_BS(BS2)
I.add_BS(BS3)

In [3]:
U = I.calculate_transformation()

In [4]:
counts = run_interferom_simulation(U, [0,0,1], 1e6)

In [5]:
print(counts)

{'001': 0.004156, '100': 0.37485, '010': 0.620994}


In [6]:
output_states = [[1,0,0],[0,1,0],[0,0,1]]
for state in output_states:
    print(f"P {state} = {output_probability([1,0,0], state, U)}")

P [1, 0, 0] = 0.0022436490538903456
P [0, 1, 0] = 0.6227563509461099
P [0, 0, 1] = 0.37499999999999994
